## 使用tree储存“望远镜法”数据，进行数据分析

核物理实验数据具有数据量巨大，物理量多，物理量具有复杂的关联关系的特点。采用传统的数组结构，既不现实也不可取，给数据分析带来极大的困难。而root提供的tree结构，既能完整地保留所有原始信息，又能简便地分析物理量之间关系，便于在原数据的基础上进行近一步分析。把数据存成tree格式，并进行相关分析是ROOT的最核心功能，是每一个用ROOT分析数据的人必须掌握和熟悉的内容。

#### 作业说明

1. 此部分作业不列入课程要求，不计分。但tree结构的学习对于同学们后续科研工作中分析数据至关重要。建议自主完成。
2. 在作业1.2中加入将事件结构存到tree，并保存成ROOT文件的代码。
2. 在tree结构中保留zz,aa,s0e,s1e,s2e分支。(zz,aa分别为粒子的z和a)
3. 将模拟的数据以tree结构的形式保存到root文件中。


### 示例代码

In [1]:
%jsroot on
void dedx_e(TGraph *gdedx){
  ifstream in("11B-Si.txt");
  string ss;
  double a, b, e, dedx; 
  if(in.is_open()){
    getline(in,ss);
    int i=0;
    while(!in.eof())
      {
	in >> a >>b>>e>>dedx>>a>>b>>a>>b>>a>>b>>a>>b;
      	gdedx->SetPoint(i, e*12,dedx);
	i++;
      }
    in.close();
  }
}

In [2]:
void tele(){
    TGraph *gdedx=new TGraph();
    dedx_e(gdedx);
    
    double  emax=400;//MeV
    int dt[3]={142,40,304};//thickness of det. in um
    double thickness[3];
    double tsum=0;
    for(int i=0;i<3;i++) {
        tsum += dt[i];
        thickness[i]=tsum;
    }
    auto r=new TRandom3(0);
    
    //add tree stucture
   
    TFile *opf=new TFile("tele.root","recreate");
    TTree *opt=new TTree("tree","tree structure for analysis");
    // 在tree结构中添加变量分支
    Double_t se[3];//energy loss in ith det.
    Int_t zz,aa;
    //将变量分支添加到tree结构中,第一个参数为变量名称，第二个为上面定义的变量地址，第三个为变量的类型说明，D表示Double_t,I表示Int_t。
    opt->Branch("se",se,"se[3]/D");
    opt->Branch("zz",&zz,"zz/I");
    opt->Branch("aa",&aa,"aa/I");
    
    
    for(int k=0;k<50000;k++) {//k
        //初始化
        memset(se,0,sizeof(se));
        zz=0;
        aa=0;
        
        double ein=r->Rndm()*emax;//energy of incident particle.
        double e=ein;
        int i=0;
        double  eleft[3]={-1,-1,-1};//remain energy after ith det.
        while(e>0 && i<=thickness[2]) {//i
          for(int j=0;j<3;j++)//j
            if(i==thickness[j]) eleft[j]=e;
            e -= gdedx->Eval(e)*1;
            i++;
        }
        if(eleft[0]<0) se[0]=ein; //stop in d0
        else {//pass through d0
          se[0]=ein-eleft[0];
        if(eleft[1]<0) se[1]=eleft[0]-e;//stop in d1
          else {//pass through d1
                se[1]=eleft[0]-eleft[1];
                if(eleft[2]<0) se[2]=eleft[1]-e;//stop in d2
                else se[2]=eleft[1]-eleft[2];//pass through d2
              }
        }
        for(int j=0;j<3;j++) se[j]+=r->Gaus(0,0.2);//add energy fluctuation.
        
        aa=12;
        zz=6;
        // 往tree结构中填充对应分支
        opt->Fill();
    }
      // 将数据写入root文件中
    opt->Write();
    opf->Close();

}
tele()

#### 分析数据文件，望远镜法粒子鉴别

In [3]:
TFile *ipf=new TFile("tele.root");
tree->Print();

******************************************************************************
*Tree    :tree      : tree structure for analysis                            *
*Entries :    50000 : Total =         1606001 bytes  File  Size =    1147846 *
*        :          : Tree compression factor =   1.40                       *
******************************************************************************
*Br    0 :se        : se[3]/D                                                *
*Entries :    50000 : Total  Size=    1203689 bytes  File Size  =    1143375 *
*Baskets :       38 : Basket Size=      32000 bytes  Compression=   1.05     *
*............................................................................*
*Br    1 :zz        : zz/I                                                   *
*Entries :    50000 : Total  Size=     200956 bytes  File Size  =       1707 *
*Baskets :        7 : Basket Size=      32000 bytes  Compression= 117.45     *
*...................................................

In [4]:
TCanvas *c1=new TCanvas();
c1->SetLogz();
gStyle->SetPalette(1);
c1->Clear();
tree->Draw("se[0]:se[1]>>(500,-5,30,500,-5,70)","","colz");
c1->Draw();

In [5]:
c1->Clear();
tree->Draw("se[1]:se[2]","","colz");
c1->Draw();

In [6]:
c1->Clear();
tree->Draw("se[0]:se[0]+se[1]+se[2]","","colz");
c1->Draw();

#### 使用cut条件选出特定成分粒子进行分析
`Draw()`函数的第二个参数即为cut条件，根据自身物理分析的需要，可以设置不同的cut条件。

In [7]:
c1->Clear();
tree->Draw("se[0]:se[1]","se[2]<1","colz");
c1->Draw();

In [8]:
c1->Clear();
tree->Draw("se[0]:se[1]","se[2]>1","colz");
c1->Draw();

##### 可以使用多个cut条件

In [9]:
c1->Clear();
tree->Draw("se[0]:se[1]","se[1]>1&&se[2]<1","colz");
c1->Draw();

In [10]:
ipf->Close();